# ConIndVar

- Concat Independent Variable
- 독립변수 합치기

- 대상 변수: 사용할 변수 목록.xlsx 참고


해야할 일

1. 결측값 대치
    1. 안 하기! XGBoost는 결측값에 강건하다
    2. missingpy 라이브러리의 missforest 사용
    3. xgbimputer 라이브러리 쓰기!
        - 이 방법 쓰려면 좀 복잡할듯
    4. 원래 하던대로 하기(최빈값, 평균값 등등)

2. 이상치 제거...?
    1. 한다
    2. 안 한다

3. 데이터 가공하기. 고혈압 여부 등





In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc

# 기본 글꼴을 D2Coding ligature으로 변경
rc("font", family="D2Coding ligature")

In [37]:
print(plt.rcParams['font.family'])

['D2Coding ligature']


- 원본 데이터 불러오기

- GH_9622 데이터셋 기준으로 인덱스 조정해야함.

In [2]:
AS1_01_EXAMINEE = pd.read_csv("./Dataset/AS1_01_EXAMINEE.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_02_GEN = pd.read_csv("./Dataset/AS1_02_GEN.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_03_DRSM = pd.read_csv("./Dataset/AS1_03_DRSM.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_04_ACTIVE = pd.read_csv("./Dataset/AS1_04_ACTIVE.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_05_MEDIC = pd.read_csv("./Dataset/AS1_05_MEDIC.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_06_DISEASE = pd.read_csv("./Dataset/AS1_06_DISEASE.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_07_TREAT = pd.read_csv("./Dataset/AS1_07_TREAT.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_08_DRUG = pd.read_csv("./Dataset/AS1_08_DRUG.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_13_SLEEP = pd.read_csv("./Dataset/AS1_13_SLEEP.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_16_DIET = pd.read_csv("./Dataset/AS1_16_DIET.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_17_FFQWEIGHT = pd.read_csv("./Dataset/AS1_17_FFQWEIGHT.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_18_BIOCHEM = pd.read_csv("./Dataset/AS1_18_BIOCHEM.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_19_ANTHRO = pd.read_csv("./Dataset/AS1_19_ANTHRO.csv", index_col=0, encoding='utf-8', low_memory=False)
GH = pd.read_csv("./Dataset/GH_9622.csv", index_col=0, encoding='utf-8', low_memory=False) # 경훈이가 가공해준 식이패턴 데이터             


## Data Engineering

### Imputation

해야할 일

- 77777 -> 0 또는 특정 값
- 99999 -> NaN

### Data 가공하기

- 고혈압여부: HAS_HYPERTENSION
- 당뇨여부: HAS_DIABETES
- 고지혈증: HAS_HYPERLIPIDEMI
- LDL: LDL
- H0MA_IR: H0MA_IR
- 허리둘레: WAIST_AVG
- BMI: BMI


In [3]:
processed_df = pd.DataFrame(index=AS1_01_EXAMINEE.index)
processed_df.head()

""
DIST_ID
NIH23B1298125
NIH23B1159376
NIH23B1785393
NIH23B1463054
NIH23B1751168


#### 고혈압여부: HAS_HYPERTENSION
> pdht == 2 or drughtcu == 2 or 3회평균_sys >= 140 or 3회평균_dia >=90

In [4]:
pdht = AS1_06_DISEASE[["AS1_PDHT"]]
drughtcu = AS1_08_DRUG[['AS1_DRUGHTCU']]
sys = AS1_19_ANTHRO[['AS1_BPLIE1S', 'AS1_BPLIE2S', 'AS1_BPLIE3S']].mean(axis=1)
dia = AS1_19_ANTHRO[['AS1_BPLIE1D', 'AS1_BPLIE2D', 'AS1_BPLIE3D']].mean(axis=1)

df = pd.concat([pdht, drughtcu], axis=1)
df["AS1_AVG_SYS"] = sys
df["AS1_AVG_DIA"] = dia
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10030 entries, NIH23B1298125 to NIH23B1160138
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   AS1_PDHT      10030 non-null  int64  
 1   AS1_DRUGHTCU  10030 non-null  int64  
 2   AS1_AVG_SYS   10030 non-null  float64
 3   AS1_AVG_DIA   10030 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 391.8+ KB


In [5]:
df["AS1_DRUGHTCU"].value_counts()

AS1_DRUGHTCU
77777    8256
2        1086
99999     619
1          69
Name: count, dtype: int64

In [6]:
conditions = (
    (df['AS1_PDHT'] == 2) | 
    (df['AS1_DRUGHTCU'] == 2) | 
    (df['AS1_AVG_SYS'] >= 140) | 
    (df['AS1_AVG_DIA'] >= 90)
)

# df['HAS_HYPERTENSION'] = np.where(conditions, 1, 0)
processed_df['HAS_HYPERTENSION'] = conditions.astype(int)

In [7]:
processed_df['HAS_HYPERTENSION'].value_counts()

HAS_HYPERTENSION
0    7429
1    2601
Name: count, dtype: int64

#### 당뇨여부

> pddm == 2 or druginscu == 2 or glu0 >= 126 or hba1c >= 6.5 or glu120 >= 200

In [8]:
pddm = AS1_06_DISEASE[["AS1_PDDM"]]
druginscu = AS1_08_DRUG[['AS1_DRUGINSCU']]
glu0 = AS1_18_BIOCHEM[['AS1_GLU0_TR']]
glu120 = AS1_18_BIOCHEM[['AS1_GLU120_TR']]
hba1c = AS1_18_BIOCHEM[['AS1_HBA1C']]

df = pd.concat([pddm, druginscu, glu0, glu120, hba1c], axis=1)

In [9]:
conditions = (
    (df['AS1_PDDM'] == 2) | 
    (df['AS1_DRUGINSCU'] == 2) | 
    (df['AS1_GLU0_TR'] >= 126) | 
    (df['AS1_GLU120_TR'] >= 200) |
    (df['AS1_HBA1C'] >= 6.5)
)

processed_df['HAS_DIABETES'] = conditions.astype(int)

In [10]:
processed_df['HAS_DIABETES'].value_counts()

HAS_DIABETES
0    8524
1    1506
Name: count, dtype: int64

#### 고지혈증
> tchl >= 240 or LDL >= 160 or TG >= 200 or HDL < 40

- 현대인은 모두 고지혈증인가? 어떻게 2가 없지

In [11]:
pdlp = AS1_06_DISEASE[["AS1_PDLP"]]
drugslcu = AS1_08_DRUG[['AS1_DRUGSLCU']]
tchl = AS1_18_BIOCHEM[['AS1_TCHL_TR']]
hdl = AS1_18_BIOCHEM[['AS1_HDL_TR']]
tg = AS1_18_BIOCHEM[['AS1_TG_TR']]

df = pd.concat([pdlp, drugslcu, tchl, hdl, tg], axis=1)
df['AS1_LDL_TR'] = df['AS1_TCHL_TR'] - df['AS1_HDL_TR'] - df['AS1_TG_TR'] / 5

In [12]:
df['AS1_TG_TR']/5

DIST_ID
NIH23B1298125    39.6
NIH23B1159376    62.0
NIH23B1785393    66.8
NIH23B1463054    21.0
NIH23B1751168    19.0
                 ... 
NIH23B1761277    22.2
NIH23B1751514    22.8
NIH23B1102799    46.4
NIH23B1964351    39.8
NIH23B1160138    38.4
Name: AS1_TG_TR, Length: 10030, dtype: float64

In [13]:
df.head()

,AS1_PDLP,AS1_DRUGSLCU,AS1_TCHL_TR,AS1_HDL_TR,AS1_TG_TR,AS1_LDL_TR
DIST_ID,,,,,,
NIH23B1298125,1,77777,159,36,198,83.4
NIH23B1159376,1,99999,156,30,310,64.0
NIH23B1785393,1,77777,161,44,334,50.2
NIH23B1463054,1,77777,161,40,105,100.0
NIH23B1751168,1,99999,151,44,95,88.0


In [14]:
conditions = (
    (df['AS1_PDLP'] == 2) | 
    (df['AS1_DRUGSLCU'] == 2) | 
    (df['AS1_TCHL_TR'] >= 240) | 
    (df['AS1_LDL_TR'] >= 160) |
    (df['AS1_TG_TR'] >= 200) |
    (df['AS1_HDL_TR']) < 40
)

processed_df['HAS_HYPERLIPIDEMI'] = conditions.astype(int)


In [15]:
processed_df['HAS_HYPERLIPIDEMI'].value_counts()

HAS_HYPERLIPIDEMI
1    10030
Name: count, dtype: int64

#### HOMA_IR, 허리둘레, BMI
> HOMA_IR=fasting glucose (mg/dL) X fasting insulin (mU/L) / 405

- AS1_Glu0_TR: mg/dL
- AS1_Ins0: µIU/㎖

- AS1_INS0 µIU/㎖ == uU/mL == mU/L
    - µ == u // 일부 이렇게 표시하는 사람 있음
    - IU == U // 같은 거임
    - 둘은 같다. 근거: https://www.quora.com/Any-one-can-help-how-I-can-convert-between-uU-ml-to-mcunit-ml-for-insulin-measuring-to-use-it-in-HOMA-IR-calculation

In [20]:
# HOMA-IR
processed_df['HOMA_IR'] = (AS1_18_BIOCHEM['AS1_GLU0_TR'] * AS1_18_BIOCHEM['AS1_INS0']) / 405

In [42]:
pd.options.display.max_rows = None
print(len(df[AS1_18_BIOCHEM['AS1_GLU0_TR'] == 99999]))
print(len(df[AS1_18_BIOCHEM['AS1_INS0'] == 99999]))

299
296


In [29]:
# 허리둘레

processed_df['WAIST_AVG'] = AS1_19_ANTHRO[['AS1_WAIST1', 'AS1_WAIST2', 'AS1_WAIST3']].mean(axis=1)

In [30]:
# BMI

processed_df['BMI'] = ((AS1_19_ANTHRO['AS1_HEIGHT'] / 100) ** 2) * AS1_19_ANTHRO['AS1_WEIGHT']

In [31]:
processed_df['HOMA_IR'].describe()

count    1.003000e+04
mean     7.286664e+05
std      4.178770e+06
min      1.654321e-02
25%      1.057778e+00
50%      1.478025e+00
75%      2.110370e+00
max      2.469086e+07
Name: HOMA_IR, dtype: float64